In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torchvision
# from torch.utils.tensorboard import SummaryWriter

In [94]:
ngpu = 1
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

In [95]:
ix_to_w = open('word_labels.txt', 'r', encoding='utf-8').read().splitlines()
ix_to_w[:6]

[',', 'de', 'que', 'el', 'la', 'a']

In [96]:
w_to_ix = {w: ix for ix, w in enumerate(ix_to_w)}
vocab_size = len(w_to_ix)
vocab_size

7796

In [97]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm) #, bias=True)#, batch_first=True)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out) #, bias=True)

    def forward(self, sentence):
        T = len(sentence)

        embeddings = self.embedding(sentence).view(T, 1, -1)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)

        preact_out = self.linear(lstm_out).view(T, -1)

        return F.log_softmax(preact_out, dim=1)

In [98]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 32 # 32

# Dimensión de la capa lstm
D_lstm = 16 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 300

# Betas para Adam
beta1 = 0.0001
beta2 = 0.99

# Learning rate
lr =  0.1

In [99]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out)
model.load_state_dict(torch.load('model-seed42069'))
model.to(device)
model.eval()

Model(
  (embedding): Embedding(7796, 32)
  (lstm): LSTM(32, 16)
  (linear): Linear(in_features=16, out_features=7796, bias=True)
)

In [133]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

In [134]:
test_sents = [
    'andrés_manuel_lópez_obrador',
    'lópez_obrador',
    'ricardo_anaya',
    'anaya',
    'josé_antonio_meade',
    'meade',
    'el candidato'
]

In [135]:
test_sents_ix = [
    [ixBOS]+[w_to_ix[w] for w in sent.split()] 
    for sent in test_sents
]
test_sents_ix

[[7793, 64],
 [7793, 43],
 [7793, 51],
 [7793, 170],
 [7793, 44],
 [7793, 59],
 [7793, 3, 25]]

In [136]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.LongTensor(l).to(device)
        for l in list_of_lists
    ]

In [137]:
X_test = to_pytorch_tensor(test_sents_ix)
X_test

[tensor([7793,   64], device='cuda:0'),
 tensor([7793,   43], device='cuda:0'),
 tensor([7793,   51], device='cuda:0'),
 tensor([7793,  170], device='cuda:0'),
 tensor([7793,   44], device='cuda:0'),
 tensor([7793,   59], device='cuda:0'),
 tensor([7793,    3,   25], device='cuda:0')]

In [138]:
def print_ix_sentence(sentence, end='\n'):
    print(' '.join(ix_to_w[ix] for ix in sentence.data), end=end)

In [139]:
def generar_sent(sentence, limit = 35, choose_max=True):
    if len(sentence) == 0:
        sentence = torch.LongTensor([ixBOS]).to(device)
    torchEOS = torch.LongTensor([ixEOS]).to(device)
    i = 0
    prediccion = sentence[-1:]
    while not torch.eq(prediccion, torchEOS):
        out = model(sentence)
        if choose_max:
            prediccion = torch.argmax(out[-1:], dim=1)
        else:
            prediccion = torch.multinomial(torch.exp(out[-1]), 1)
#         print_ix_sentence(prediccion, end=' ') # Imprimo el siguiente caracter
        sentence = torch.cat((sentence, prediccion), dim=0) # Genero siguiente cadena
        i+=1
        if i>limit:
            break
    return sentence

In [140]:
with torch.no_grad():
    for sentence in X_test[:10]:
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()
        
        print('- Enunciado generado:')
        gen_sentence = generar_sent(sentence, limit=60)
        print_ix_sentence(gen_sentence)
        print('\n')

#         out = model(sentence)
#         prediccion = torch.argmax(out[-1:], dim=1)
#         print('- Texto generado:')
#         print_ix_sentence(prediccion)

- Probando enunciado:
<BOS> andrés_manuel_lópez_obrador

- Enunciado generado:
<BOS> andrés_manuel_lópez_obrador expresó que en los últimos días , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país ,


- Probando enunciado:
<BOS> lópez_obrador

- Enunciado generado:
<BOS> lópez_obrador expresó que en la <UNK> de el país , no se <UNK> , a el país , porque se <UNK> a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque


- Probando enunciado:
<BOS> ricardo_anaya

- Enunciado generado:
<BOS> ricardo_anaya estuvo en <UNK> , josé_antonio_meade , dijo , josé_antonio_meade , dijo . <EOS>


- Probando enunciado:
<BOS> anaya

- Enunciado generado:
<BOS> anaya no se <UNK> , a el país , porque no se <UNK> , a el país , porque no se <UNK> , a el país , porque n

In [141]:
torch.exp(out[-1]).sum()

tensor(1.0000, device='cuda:0')

In [148]:
torch.multinomial(torch.exp(out[-1]), 1)

tensor([16], device='cuda:0')